In [ ]:



import pandas as pd
import numpy as np
import random
import time
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
import joblib
from web3 import Web3
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State
import dash
from dash import Dash, dcc, html, Input, Output, State, no_update
import dash_bootstrap_components as dbc
import networkx as nx
import matplotlib.pyplot as plt
import socket
import signal
import sys
import logging
import os
import io
import base64
from PIL import Image
import google.generativeai as genai





In [ ]:
# Setup logging for tracking anomalies
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


API_KEY =  ""
genai.configure(api_key=API_KEY)

In [2]:



# Blockchain Configuration
BLOCKCHAIN_API_KEY = "e65ee1b207274346b6a586c24e43bb18"
BLOCKCHAIN_URL = f"https://mainnet.infura.io/v3/{BLOCKCHAIN_API_KEY}"
w3 = Web3(Web3.HTTPProvider(BLOCKCHAIN_URL))

class BlockchainLedger:
    def __init__(self):
        self.verified_addresses = set([
            '0x742d35Cc6634C0532925a3b844Bc454e4438f44e', 
            '0x1aD91ee08f21bE3dE0BA2ba6918E714dA6B45836'
        ])
        self.w3 = w3
        
    def is_connected(self):
        return True

    def verify_address(self, address):
        return Web3.is_address(address) and address in self.verified_addresses

    def is_verified(self, address):
        return self.verify_address(address)

ledger = BlockchainLedger()

In [3]:

# Data Storage and Blocked Accounts
transaction_history = []
alerts = []
blocked_accounts = set()
anomalies_detected = []
llm_insights = ""

In [4]:

# Function to automatically block/freeze accounts
def block_account(address):
    global blocked_accounts
    if address not in blocked_accounts:
        blocked_accounts.add(address)
        logger.warning(f"Account {address} has been automatically blocked/frozen.")

In [5]:
# Simulate Real-Time Transaction
def simulate_real_time_transaction():
    address = random.choice([
        Web3.to_checksum_address(f"0x{random.randbytes(20).hex()}"),
        random.choice([
            '0x742d35Cc6634C0532925a3b844Bc454e4438f44e', 
            '0x1aD91ee08f21bE3dE0BA2ba6918E714dA6B45836'
        ])
    ])
    transaction = {
        'Sent tnx': random.randint(1, 10),
        'Received Tnx': random.randint(1, 10),
        'avg val sent': random.uniform(0.01, 5.0),
        'avg val received': random.uniform(0.01, 5.0),
        'Unique Sent To Addresses': random.randint(1, 20),
        'Unique Received From Addresses': random.randint(1, 20),
        'Time Diff between first and last (Mins)': random.uniform(1, 120),
        'Timestamp': datetime.now(),
        'Account Age': random.randint(1, 100),
        'Device Fingerprint': random.choice(['DeviceA', 'DeviceB', 'DeviceC']),
        'IP Address': random.choice(['192.168.0.1', '192.168.0.2']),
        'Geolocation': random.choice(['US', 'EU', 'Asia']),
        'Address': address
    }
    transaction_history.append(transaction)
    return transaction

In [6]:
# Feature Engineering Functions
def feature_engineering_real_time_fake(transaction_data, previous_data=None):
    time_diff = (transaction_data['Timestamp'] - previous_data['Timestamp']).total_seconds() / 60.0 if previous_data else 0
    return pd.DataFrame([{
        'Transaction Frequency': transaction_data['Sent tnx'] + transaction_data['Received Tnx'],
        'Average Sent Amount': transaction_data['avg val sent'],
        'Average Received Amount': transaction_data['avg val received'],
        'Unique Sent Addresses': transaction_data['Unique Sent To Addresses'],
        'Unique Received Addresses': transaction_data['Unique Received From Addresses'],
        'Transaction Time Consistency': transaction_data['Time Diff between first and last (Mins)'],
        'Time Diff between Transactions (Minutes)': time_diff
    }])

In [7]:
def feature_engineering_real_time_sybil(transaction_data, previous_data=None):
    time_diff = (transaction_data['Timestamp'] - previous_data['Timestamp']).total_seconds() / 60.0 if previous_data else 0
    return pd.DataFrame([{
        'Transaction Frequency': transaction_data['Sent tnx'] + transaction_data['Received Tnx'],
        'Average Sent Amount': transaction_data['avg val sent'],
        'Average Received Amount': transaction_data['avg val received'],
        'Unique Sent Addresses': transaction_data['Unique Sent To Addresses'],
        'Unique Received Addresses': transaction_data['Unique Received From Addresses'],
        'Transaction Time Consistency': transaction_data['Time Diff between first and last (Mins)'],
        'Time Diff between Transactions (Minutes)': time_diff,
        'Account Age': transaction_data['Account Age'],
        'Total Sent Transactions': transaction_data['Sent tnx'],
        'Total Received Transactions': transaction_data['Received Tnx'],
        'Device Fingerprint': transaction_data['Device Fingerprint'],
        'IP Address': transaction_data['IP Address'],
        'Geolocation': transaction_data['Geolocation']
    }])

def feature_engineering_real_time_bot(transaction_data, previous_data=None):
    time_diff = (transaction_data['Timestamp'] - previous_data['Timestamp']).total_seconds() / 60.0 if previous_data else 0
    variance = np.std([transaction_data['avg val sent'], transaction_data['avg val received']])
    frequency = transaction_data['Sent tnx'] + transaction_data['Received Tnx']
    return pd.DataFrame([{
        'Transaction Time Diff': transaction_data['Time Diff between first and last (Mins)'],
        'Transaction Amount Variance': variance,
        'Unique Sent Addresses': transaction_data['Unique Sent To Addresses'],
        'Bot Activity Indicator': frequency * variance * transaction_data['Unique Sent To Addresses']
    }])


In [8]:
pipeline_fake = joblib.load('fake_identities_model.pkl')
pipeline_sybil = joblib.load('sybil_attacks_model.pkl')
pipeline_bot = joblib.load('bot_activity_model.pkl')

## Alert Detection Logic with Automatic Blocking

In [9]:

# Alert Detection Logic with Automatic Blocking
def detect_alerts(transaction_data, previous_data=None):
    global alerts, blocked_accounts
    current_alerts = []
    address = transaction_data['Address']
    
    # Skip further processing if the account is already blocked
    if address in blocked_accounts:
        current_alerts.append(f"Account {address} is already blocked.")
        alerts.extend(current_alerts)
        return

    df = pd.DataFrame(transaction_history)
    
    # Check for potential Sybil Attack clusters based on IP or device fingerprint
    if len(df) > 1:
        similar_ip = df[df['IP Address'] == transaction_data['IP Address']]
        similar_device = df[df['Device Fingerprint'] == transaction_data['Device Fingerprint']]
        if len(similar_ip) > 3 or len(similar_device) > 3:
            current_alerts.append(f"Sybil Attack detected - Account {address} has been automatically blocked.")
            block_account(address)
    
    # Calculate features for all models
    features_fake = feature_engineering_real_time_fake(transaction_data, previous_data)
    features_sybil = feature_engineering_real_time_sybil(transaction_data, previous_data)
    features_bot = feature_engineering_real_time_bot(transaction_data, previous_data)
    
    # Fake Identity Detection (investigation only)
    freq = features_fake['Transaction Frequency'].iloc[0]
    avg_sent = features_fake['Average Sent Amount'].iloc[0]
    time_diff = features_fake['Time Diff between Transactions (Minutes)'].iloc[0]
    if freq > 5 and avg_sent < 1.0 and (time_diff < 5 or time_diff == 0):
        current_alerts.append(f"Fake Identity detected for account {address}. Investigation recommended.")
    
    # Bot Detection - automatically block account
    bot_indicator = features_bot['Bot Activity Indicator'].iloc[0]
    if bot_indicator > 50:
        current_alerts.append(f"Bot Activity detected - Account {address} has been automatically blocked.")
        block_account(address)
    
    # Model-based Predictions for Sybil and Bot
    if pipeline_fake and pipeline_sybil and pipeline_bot:
        if pipeline_fake.predict(features_fake)[0] == 1:
            current_alerts.append(f"Fake Identity (Model): Investigation recommended for account {address}.")
        if pipeline_sybil.predict(features_sybil)[0] == 1:
            current_alerts.append(f"Sybil Attack (Model) detected - Account {address} has been automatically blocked.")
            block_account(address)
        if pipeline_bot.predict(features_bot)[0] == 1:
            current_alerts.append(f"Bot Activity (Model) detected - Account {address} has been automatically blocked.")
            block_account(address)
    
    # Unverified address handling (not auto-blocked)
    if not ledger.is_verified(address):
        current_alerts.append(f"Unverified Address: Blockchain check failed for account {address}. Follow-up action recommended.")
    
    if current_alerts:
        alerts.extend(current_alerts)

In [10]:
# New Graph-based Anomaly Detection Functions

# Function to simulate blockchain wallet data from existing transactions
def create_wallet_transactions():
    """Create wallet transaction data from existing transaction history"""
    transactions = []
    
    for tx in transaction_history:
        # Create source wallet
        src = tx['Address']
        
        # Create destination wallet(s) based on sent transactions
        for _ in range(min(tx['Sent tnx'], 3)):  # Limit to max 3 destinations for clarity
            dst = f"wallet_{random.randint(1000, 9999)}"
            amount = tx['avg val sent']
            transactions.append((src, dst, amount))
    
    wallets = list(set([t[0] for t in transactions] + [t[1] for t in transactions]))
    return wallets, transactions


# Build a directed graph from the transaction data
def build_wallet_graph(transactions):
    """
    Create a directed graph from transaction data
    
    Args:
        transactions: List of tuples in format (source, destination, amount)
        
    Returns:
        NetworkX DiGraph object
    """
    G = nx.DiGraph()
    for src, dst, amount in transactions:
        if G.has_edge(src, dst):
            # Sum up transaction amounts between the same wallets
            G[src][dst]['weight'] += amount
        else:
            G.add_edge(src, dst, weight=amount)
    return G


In [11]:
# Extract features from the graph: in-degree, out-degree, and transaction volumes
def extract_features(G):
    """
    Extract wallet features from the transaction graph
    
    Args:
        G: NetworkX DiGraph object
        
    Returns:
        Dictionary mapping wallet IDs to feature lists
    """
    features = {}
    for node in G.nodes():
        in_deg = G.in_degree(node)
        out_deg = G.out_degree(node)
        
        # Sum of incoming transaction amounts
        in_amount = sum(G[u][node]['weight'] for u in G.predecessors(node)) if in_deg > 0 else 0
        
        # Sum of outgoing transaction amounts
        out_amount = sum(G[node][v]['weight'] for v in G.successors(node)) if out_deg > 0 else 0
        
        features[node] = [in_deg, out_deg, in_amount, out_amount]
    return features

In [12]:
# Use IsolationForest for anomaly detection
def detect_graph_anomalies(features, contamination=0.1):
    """
    Detect anomalous wallets using Isolation Forest
    
    Args:
        features: Dictionary of wallet features
        contamination: Proportion of outliers in the data
        
    Returns:
        List of anomalous wallet IDs
    """
    global anomalies_detected
    nodes = list(features.keys())
    data = np.array(list(features.values()))
    
    if len(data) == 0:
        logger.warning("No data to analyze for anomalies")
        return []
    
    # Normalize features for better detection
    if np.std(data, axis=0).min() > 0:  # Avoid division by zero
        data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
    
    clf = IsolationForest(random_state=42, contamination=contamination)
    clf.fit(data)
    preds = clf.predict(data)
    # -1 indicates anomaly
    anomalies = [nodes[i] for i, pred in enumerate(preds) if pred == -1]
    
    # Add newly detected anomalies to global list and block them
    for addr in anomalies:
        if addr not in anomalies_detected:
            anomalies_detected.append(addr)
            alerts.append(f"Graph Anomaly detected for account {addr}. Automatically blocked.")
            block_account(addr)
    
    return anomalies

In [13]:
# Visualize the wallet network and highlight suspicious nodes
def visualize_graph(G, anomalies):
    """
    Visualize the transaction graph with highlighted anomalies
    
    Args:
        G: NetworkX DiGraph object
        anomalies: List of anomalous wallet IDs
        
    Returns:
        Base64 encoded PNG image
    """
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G, seed=42)
    
    # Draw normal nodes
    normal_nodes = [node for node in G.nodes() if node not in anomalies]
    nx.draw_networkx_nodes(G, pos, nodelist=normal_nodes, node_color='green', alpha=0.8, node_size=300)
    
    # Draw anomalous nodes
    if anomalies:
        nx.draw_networkx_nodes(G, pos, nodelist=anomalies, node_color='red', alpha=0.8, node_size=300)
    
    # Draw edges with width proportional to transaction amount
    edge_widths = [min(G[u][v]['weight'] * 0.5, 3) for u, v in G.edges()]  # Cap the width
    nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.5, arrows=True)
    
    # Draw labels (only for smaller graphs)
    if len(G.nodes()) < 30:
        nx.draw_networkx_labels(G, pos, font_size=8)
    
    plt.title("Wallet Network with Detected Anomalies")
    plt.axis('off')
    
    # Save to a bytes buffer instead of file
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close()
    
    # Encode to base64
    graph_base64 = base64.b64encode(buf.read()).decode('utf-8')
    return f"data:image/png;base64,{graph_base64}"

In [14]:
# Function to get LLM insights from Google Gemini
def get_llm_insights(alerts_list, anomalies_list):
    """Get security insights from Google Gemini based on alerts and anomalies"""
    global llm_insights
    
    
    try:
        prompt = "You are a blockchain security analyst. Analyze the following alerts and detected anomalies, and provide insights and recommendations.\n\n"
        prompt += "Alerts:\n" + "\n".join(str(a) for a in alerts_list) + "\n\n"
        prompt += "Detected Anomalies (Blocked Accounts): " + ", ".join(str(a) for a in anomalies_list) + "\n\n"
        prompt += "Provide a brief summary and recommendations:"
        
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        llm_insights = response.text
        return llm_insights
    except Exception as e:
        logger.error(f"Error getting LLM insights: {e}")
        return f"Error getting AI insights: {str(e)}"



##  Dashboard & Server Functions


In [15]:


# Function to Find an Available Port
def find_free_port(start_port=8050, max_attempts=10):
    port = start_port
    for _ in range(max_attempts):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                port += 1
    raise OSError(f"No free ports found between {start_port} and {start_port + max_attempts - 1}")

# Signal Handler for Clean Shutdown
def signal_handler(sig, frame):
    print("\nShutting down Dash server gracefully...")
    sys.exit(0)

# Dash Application
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Blockchain Transaction Monitoring Dashboard", className="mt-4 mb-4"),
            dcc.Interval(id='interval-component', interval=5*1000, n_intervals=0),
        ], width=12)
    ]),
    
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Alerts", className="bg-danger text-white"),
                dbc.CardBody(id='alerts-list')
            ], className="mb-4"),
            dbc.Card([
                dbc.CardHeader("Blocked Accounts", className="bg-dark text-white"),
                dbc.CardBody(id='blocked-accounts')
            ], className="mb-4")
        ], width=4),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Transaction Activity Heatmap"),
                dbc.CardBody([
                    dcc.Graph(id='heatmap')
                ])
            ], className="mb-4"),
            dbc.Card([
                dbc.CardHeader("User Interaction Network"),
                dbc.CardBody([
                    dcc.Graph(id='network-graph')
                ])
            ], className="mb-4")
        ], width=8)
    ]),
    
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Transaction Frequency and Amount Over Time"),
                dbc.CardBody([
                    dcc.Graph(id='time-series')
                ])
            ], className="mb-4")
        ], width=6),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Wallet Graph Anomaly Detection"),
                dbc.CardBody([
                    html.Img(id='graph-visualization', style={'width': '100%'})
                ])
            ], className="mb-4")
        ], width=6)
    ]),
    
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("AI Security Analysis", className="bg-primary text-white"),
                dbc.CardBody(id='ai-insights'),
                dbc.CardFooter([
                    dbc.Button("Run AI Analysis", id="run-ai-analysis", color="primary"),
                    html.Span("  Powered by Google Gemini", className="text-muted ml-2")
                ])
            ], className="mb-4")
        ], width=12)
    ])
], fluid=True)

@app.callback(
    [Output('heatmap', 'figure'),
     Output('network-graph', 'figure'),
     Output('time-series', 'figure'),
     Output('alerts-list', 'children'),
     Output('blocked-accounts', 'children'),
     Output('graph-visualization', 'src')],
    [Input('interval-component', 'n_intervals')]
)
def update_dashboard(n):
    global transaction_history, alerts, blocked_accounts, anomalies_detected
    previous_data = transaction_history[-1] if transaction_history else None
    
    if n > 0:
        # Simulate new transaction and detect rule-based alerts
        transaction_data = simulate_real_time_transaction()
        detect_alerts(transaction_data, previous_data)
        
        # Run graph-based anomaly detection every 5 iterations
        if n % 5 == 0 and len(transaction_history) > 10:
            wallets, transactions = create_wallet_transactions()
            G = build_wallet_graph(transactions)
            features = extract_features(G)
            anomalies = detect_graph_anomalies(features)
            
            # Generate wallet graph visualization
            graph_img = visualize_graph(G, anomalies)
        else:
            graph_img = "" if n == 0 else dash.no_update

    # Create dataframe from transaction history
    df = pd.DataFrame(transaction_history)
    
    if len(df) == 0:
        # Return empty figures if no data yet
        empty_fig = go.Figure()
        empty_fig.update_layout(title="No data yet")
        return empty_fig, empty_fig, empty_fig, "No alerts yet", "No blocked accounts", ""

    # Heatmap of transaction activity by geolocation
    heatmap_data = df.groupby('Geolocation').agg({
        'Sent tnx': 'sum',
        'Received Tnx': 'sum'
    }).reset_index()
    heatmap_data['Total Tnx'] = heatmap_data['Sent tnx'] + heatmap_data['Received Tnx']
    heatmap_fig = px.density_heatmap(
        heatmap_data, 
        x='Geolocation', 
        y='Total Tnx', 
        title='Transaction Activity Heatmap',
        color_continuous_scale='Viridis'
    )
    heatmap_fig.update_layout(height=300)

    # Create a user interaction network graph
    G = nx.Graph()
    for i, row in df.iterrows():
        G.add_node(row['Address'])
        if i > 0 and random.random() < 0.3:
            G.add_edge(df.iloc[i-1]['Address'], row['Address'])
    
    pos = nx.spring_layout(G)
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    node_x, node_y = [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
    
    network_fig = go.Figure()
    network_fig.add_trace(go.Scatter(x=edge_x, y=edge_y, mode='lines', line=dict(width=0.5, color='gray')))
    
    # Color nodes by whether they are blocked
    node_colors = ['red' if node in blocked_accounts else 'blue' for node in G.nodes()]
    
    network_fig.add_trace(go.Scatter(
        x=node_x, y=node_y, 
        mode='markers', 
        marker=dict(size=10, color=node_colors),
        text=list(G.nodes()),
        hoverinfo='text'
    ))
    network_fig.update_layout(title='User Interaction Network', showlegend=False, height=300)

    # Time series chart for transaction frequency and average sent amount
    df['Total Tnx'] = df['Sent tnx'] + df['Received Tnx']
    time_series_fig = go.Figure()
    time_series_fig.add_trace(go.Scatter(
        x=df['Timestamp'], 
        y=df['Total Tnx'], 
        mode='lines+markers', 
        name='Transaction Frequency'
    ))
    time_series_fig.add_trace(go.Scatter(
        x=df['Timestamp'], 
        y=df['avg val sent'], 
        mode='lines+markers', 
        name='Avg Sent Amount',
        yaxis='y2'
    ))
    time_series_fig.update_layout(
        title='Transaction Metrics Over Time',
        yaxis=dict(title='Transaction Frequency'),
        yaxis2=dict(title='Amount (ETH)', overlaying='y', side='right'),
        height=300
    )

    alert_items = [html.P(alert, className="alert alert-danger") for alert in alerts[-10:]]
    blocked_list = [html.P(acc, className="alert alert-dark") for acc in sorted(blocked_accounts)]
    
    # Create wallet graph once we have some transactions
    if n == 0 or len(transaction_history) < 5:
        graph_img = ""
    elif n % 5 == 0 or graph_img == "":
        wallets, transactions = create_wallet_transactions()
        G = build_wallet_graph(transactions)
        features = extract_features(G)
        anomalies = detect_graph_anomalies(features)
        graph_img = visualize_graph(G, anomalies)
    
    return heatmap_fig, network_fig, time_series_fig, alert_items, blocked_list, graph_img

@app.callback(
    Output('ai-insights', 'children'),
    [Input('run-ai-analysis', 'n_clicks')],
    [State('alerts-list', 'children')]
)
def update_ai_insights(n_clicks, alert_list):
    if n_clicks is None:
        return html.P("Click 'Run AI Analysis' to get security insights from the LLM.")
    
    alerts_text = [a['props']['children'] for a in alert_list] if alert_list else []
    anomalies_text = list(blocked_accounts)
    
    insights = get_llm_insights(alerts_text, anomalies_text)
    
    # Convert markdown to HTML paragraphs
    insight_paragraphs = [html.P(p) for p in insights.split('\n\n')]
    
    return insight_paragraphs

In [ ]:


# Initialize some transactions for the first load
for _ in range(10):
    tx = simulate_real_time_transaction()
    previous = transaction_history[-2] if len(transaction_history) >= 2 else None
    detect_alerts(tx, previous)

if __name__ == "__main__":
    signal.signal(signal.SIGINT, signal_handler)
    
    try:
        manual_port = None  
        port = manual_port if manual_port else find_free_port(start_port=8050)
        print(f"Starting Dash server on port {port}")
        print("Visit http://127.0.0.1:{port} to view the dashboard")
        app.run_server(debug=True, port=port, use_reloader=False)
    except OSError as e:
        print(f"Failed to start server: {e}")
        print("Please try one of the following:")
        print("1. Run the script again to try a different port.")
        print("2. Manually specify a port by setting 'manual_port' (e.g., 8051).")
        print("3. Stop existing Dash servers using the steps below:")
        print("   - Windows: Task Manager -> End 'python.exe' processes")
        print("   - Linux/Mac: 'lsof -i :8050' then 'kill -9 <PID>'")

2025-03-01 00:14:13,070 - WARNING - Account 0xffCC6b8069b25b849BA35fBC9Bf86a9DD4EEa940 has been automatically blocked/frozen.
2025-03-01 00:14:13,391 - WARNING - Account 0x742d35Cc6634C0532925a3b844Bc454e4438f44e has been automatically blocked/frozen.
2025-03-01 00:14:13,483 - WARNING - Account 0x2289260A9ed9e3c0671E17e3c417AaE6c990024c has been automatically blocked/frozen.
2025-03-01 00:14:13,584 - WARNING - Account 0x1aD91ee08f21bE3dE0BA2ba6918E714dA6B45836 has been automatically blocked/frozen.
2025-03-01 00:14:13,692 - WARNING - Account 0x748E2945285d36a40f28e94CC6C4b10230df9B2C has been automatically blocked/frozen.


Starting Dash server on port 8050
Visit http://127.0.0.1:{port} to view the dashboard


2025-03-01 00:14:19,699 - WARNING - Account 0xdd04b904A6f338152aeAB6e09aEC8585335658d5 has been automatically blocked/frozen.
2025-03-01 00:14:39,486 - WARNING - Account 0x9415a5BB3b3f5a614d9b449E7395722f849DAb7c has been automatically blocked/frozen.
2025-03-01 00:14:44,258 - WARNING - Account 0xA8284a60aAEF0e54350Af8B5004540DEd63fB45f has been automatically blocked/frozen.
2025-03-01 00:14:49,258 - WARNING - Account 0xbB5d6125cbD7092974bC6F5B0C1216d902d669a5 has been automatically blocked/frozen.
2025-03-01 00:14:59,281 - WARNING - Account 0x65bE2F82e7D314460388B0E7ece4c24bADd4dbFb has been automatically blocked/frozen.
2025-03-01 00:15:04,260 - WARNING - Account 0xbeEDafe1A9cCA0235F51B3255D2aCA086Cc13F4E has been automatically blocked/frozen.
2025-03-01 00:15:04,556 - WARNING - Account wallet_5066 has been automatically blocked/frozen.
2025-03-01 00:15:04,557 - WARNING - Account wallet_4509 has been automatically blocked/frozen.
2025-03-01 00:15:04,558 - WARNING - Account wallet_283

---------------------------------------------------------------------------
MethodNotAllowed                          Traceback (most recent call last)
MethodNotAllowed: 405 Method Not Allowed: The method is not allowed for the requested URL.



2025-03-01 01:39:14,014 - WARNING - Account 0x6A17925e57B40E73552415D00BF67fE1481F94d0 has been automatically blocked/frozen.
2025-03-01 01:39:14,865 - WARNING - Account 0xC2d8e61536DED977EDcb6Fa6038C27Bd2c3549B0 has been automatically blocked/frozen.
2025-03-01 01:39:29,411 - WARNING - Account 0xfcEc3E3C6d2730a835eC1ACd44A9464250714312 has been automatically blocked/frozen.
2025-03-01 01:39:30,309 - WARNING - Account wallet_4268 has been automatically blocked/frozen.
2025-03-01 01:39:30,312 - WARNING - Account wallet_6381 has been automatically blocked/frozen.
2025-03-01 01:39:30,316 - WARNING - Account wallet_7081 has been automatically blocked/frozen.
2025-03-01 01:39:30,321 - WARNING - Account wallet_2892 has been automatically blocked/frozen.
2025-03-01 01:39:30,426 - WARNING - Account wallet_1485 has been automatically blocked/frozen.
2025-03-01 01:39:30,470 - WARNING - Account wallet_2867 has been automatically blocked/frozen.
2025-03-01 01:39:30,505 - WARNING - Account wallet_4

...

2025-03-01 03:02:53,535 - WARNING - Account wallet_4740 has been automatically blocked/frozen.
2025-03-01 03:02:53,552 - WARNING - Account wallet_8003 has been automatically blocked/frozen.
2025-03-01 03:02:53,560 - WARNING - Account wallet_7694 has been automatically blocked/frozen.
2025-03-01 03:02:53,570 - WARNING - Account wallet_7438 has been automatically blocked/frozen.
2025-03-01 03:02:53,574 - WARNING - Account wallet_2481 has been automatically blocked/frozen.
2025-03-01 03:02:53,585 - WARNING - Account wallet_7570 has been automatically blocked/frozen.
2025-03-01 03:02:53,594 - WARNING - Account wallet_1340 has been automatically blocked/frozen.
2025-03-01 03:03:13,455 - WARNING - Account 0x132586d83A4e347c426e61972c08BC209Fa0D657 has been automatically blocked/frozen.
2025-03-01 03:03:15,054 - WARNING - Account 0xd66ACf99aC13d910340FeFCD66843266262A4316 has been automatically blocked/frozen.
2025-03-01 03:03:16,702 - WARNING - Account 0xE7de6B2142a9e4a1A0dC29A2DD72cF16F25EB

---------------------------------------------------------------------------
MethodNotAllowed                          Traceback (most recent call last)
MethodNotAllowed: 405 Method Not Allowed: The method is not allowed for the requested URL.

---------------------------------------------------------------------------
MethodNotAllowed                          Traceback (most recent call last)
MethodNotAllowed: 405 Method Not Allowed: The method is not allowed for the requested URL.



2025-03-01 03:10:51,169 - WARNING - Account 0x674d569B39ac4d11258e2ffA046C552Dd9c4Ec63 has been automatically blocked/frozen.


---------------------------------------------------------------------------
MethodNotAllowed                          Traceback (most recent call last)
MethodNotAllowed: 405 Method Not Allowed: The method is not allowed for the requested URL.



2025-03-01 03:10:57,315 - WARNING - Account wallet_5074 has been automatically blocked/frozen.
2025-03-01 03:10:57,321 - WARNING - Account wallet_9854 has been automatically blocked/frozen.
2025-03-01 03:10:57,323 - WARNING - Account wallet_2886 has been automatically blocked/frozen.
2025-03-01 03:10:57,329 - WARNING - Account wallet_3902 has been automatically blocked/frozen.
2025-03-01 03:10:57,335 - WARNING - Account wallet_3072 has been automatically blocked/frozen.
2025-03-01 03:10:57,344 - WARNING - Account wallet_4625 has been automatically blocked/frozen.
2025-03-01 03:10:57,352 - WARNING - Account wallet_3364 has been automatically blocked/frozen.
2025-03-01 03:10:57,364 - WARNING - Account wallet_1563 has been automatically blocked/frozen.
2025-03-01 03:10:57,373 - WARNING - Account wallet_8847 has been automatically blocked/frozen.
2025-03-01 03:10:57,384 - WARNING - Account wallet_1840 has been automatically blocked/frozen.
2025-03-01 03:10:57,393 - WARNING - Account wallet